In [1]:
!pip install "tensorflow-text==2.13.*"

ERROR: Could not find a version that satisfies the requirement tensorflow-text==2.13.* (from versions: none)
ERROR: No matching distribution found for tensorflow-text==2.13.*


In [ ]:
!pip install "tf-models-official==2.13.*"

In [1]:
import pandas as pd
import numpy as np
import re
import spacy


In [2]:
dataset=pd.read_csv("Evaluation-dataset.csv")

In [3]:
col={"Review":1}
for i in range(1,len(dataset.axes[0])):
    for j in range(1, len(dataset.axes[1])):
        if 'positive' in str(dataset.loc[i][j]):
            sentiment=str(dataset.loc[i][j])
            col[sentiment[:-9]]=1
        if 'negative' in str(dataset.loc[i][j]):
            sentiment=str(dataset.loc[i][j])
            col[sentiment[:-9]]=1

In [4]:
newdata=pd.DataFrame(columns=[i for i in col])

In [5]:
newdata["Review"]=dataset["review"]

In [6]:
col={"Review":1}
for i in range(0,len(dataset.axes[0])):
    for j in range(1, len(dataset.axes[1])):
        if 'positive' in str(dataset.loc[i][j]):
            sentiment=str(dataset.loc[i][j])
            newdata[sentiment[:-9]][i]=1
        if 'negative' in str(dataset.loc[i][j]):
            sentiment=str(dataset.loc[i][j])
            newdata[sentiment[:-9]][i]=-1

In [8]:
newdata.fillna(0, inplace=True)
newdata

,Review,garage service,value for money,ease of booking,location,length of fitting,change of date,tyre quality,wait time,delivery punctuality,...,mobile fitter didn't arrive,discount not applied,tyre agedot code,failed payment,incorrect tyres sent,call wait time,refund,no stock,response time,tyre age/dot code
0,Tires where delivered to the garage of my choi...,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Easy Tyre Selection Process, Competitive Prici...",1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Very easy to use and good value for money.,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Really easy and convenient to arrange,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,It was so easy to select tyre sizes and arrang...,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10127,"I ordered the wrong tyres, however [REDACTED] ...",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
10128,"Good experience, first time I have used [REDAC...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10129,"I ordered the tyre I needed on line, booked a ...",0,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10130,Excellent service from point of order to fitti...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def get_labels(df):
    final=[]
    outshape=[]
    for i in df.columns:
        Y=df[i]
        Y = pd.get_dummies(Y)
        Y = Y.values
        final.append(Y)
        outshape.append(Y.shape[0])

    return final, outshape

In [10]:
x=newdata["Review"]
y=newdata.drop("Review", axis=1)
final, outshape = get_labels(y)

In [11]:
y.columns

Index(['garage service', 'value for money', 'ease of booking', 'location',
       'length of fitting', 'change of date', 'tyre quality', 'wait time',
       'delivery punctuality', 'mobile fitter', 'advisor/agent service',
       'advisoragent service', 'extra charges', 'damage', 'balancing',
       'facilities', 'change of time', 'booking confusion', 'late notice',
       'discounts', 'refund not actioned', 'refund timescale',
       'mobile fitter didn't arrive', 'discount not applied',
       'tyre agedot code', 'failed payment', 'incorrect tyres sent',
       'call wait time', 'refund', 'no stock', 'response time',
       'tyre age/dot code'],
      dtype='object')

In [12]:
final[0].shape

(10132, 3)

In [13]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
# bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
# bert_model = hub.KerasLayer(tfhub_handle_encoder)

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
tfhub_handle_encoder='https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1'
tfhub_handle_preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

def build_classifier_model(outshape):
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  out_layers=[]
  for i in outshape:
    out = tf.keras.layers.Dropout(0.1)(net)
    out = tf.keras.layers.Dense(8, activation="relu")(out)
    out = tf.keras.layers.Dense(i, activation="softmax",)(out)
    out_layers.append(out)

  return tf.keras.Model(text_input, outputs=out_layers)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
losses=[]
metrics=[]
for i in outshape:
  losses.append("categorical_crossentropy")
  metrics.append("accuracy")
model=build_classifier_model(outshape)
model.compile(loss=losses, optimizer=optimizer, metrics=metrics)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
history=model.fit(x, final,
                  epochs=50,
                  batch_size=32)

In [ ]:
history